In [ ]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np

def compute_fashion_mnist_mean_std(root="./data"):
    """
    Load raw FashionMNIST training data, convert to float in [0,1],
    and compute global mean and std over all pixels, as recommended in CS231n:
    center data to mean 0 and normalize its scale.
    """
    # Load once without transforms to access raw uint8 data
    raw_train = datasets.FashionMNIST(
        root=root,
        train=True,
        download=True,
        transform=None
    )

    # raw_train.data: shape [60000, 28, 28], dtype uint8 in [0, 255]
    train_data = raw_train.data.float() / 255.0 # match ToTensor scaling

    mean = train_data.mean().item()
    std = train_data.std().item()
    return mean, std


def get_fashion_mnist_datasets(root="./data", val_ratio=0.2, seed=551):
    """
    Acquire FashionMNIST, compute normalization statistics on training set,
    and return normalized train, validation, and test datasets.

    - Uses the default 28x28 version.
    - Uses the 60k official training split for train + validation.
    - Uses the 10k official test split as test.
    """
    mean, std = compute_fashion_mnist_mean_std(root)

    train_transform = transforms.Compose([
        transforms.ToTensor(), # [0, 255] -> [0, 1]
        transforms.Normalize((mean,), (std,)) # zero mean, unit-ish variance
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean,), (std,))
    ])

    full_train_dataset = datasets.FashionMNIST(
        root=root,
        train=True,
        download=True,
        transform=train_transform
    )

    test_dataset = datasets.FashionMNIST(
        root=root,
        train=False,
        download=True,
        transform=test_transform
    )

    # Split 60k training samples into train and validation
    total_train = len(full_train_dataset) # should be 60000
    val_size = int(val_ratio * total_train)
    train_size = total_train - val_size

    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset = random_split(
        full_train_dataset,
        [train_size, val_size],
        generator=generator
    )

    return train_dataset, val_dataset, test_dataset, mean, std


def get_fashion_mnist_loaders(
    root="./data",
    val_ratio=0.2,
    batch_size=128,
    num_workers=2,
    seed=551
):
    """
    Convenience function that wraps dataset acquisition and returns
    DataLoaders for train, validation, and test sets.
    """
    train_dataset, val_dataset, test_dataset, mean, std = get_fashion_mnist_datasets(
        root=root,
        val_ratio=val_ratio,
        seed=seed
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, val_loader, test_loader, mean, std


if __name__ == "__main__":
    train_loader, val_loader, test_loader, mean, std = get_fashion_mnist_loaders()

    print(f"Train batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print(f"Test batches: {len(test_loader)}")
    print(f"Computed mean: {mean:.4f}, std: {std:.4f}")

    # Inspect one batch shape
    images, labels = next(iter(train_loader))
    # images shape: [batch_size, 1, 28, 28]
    print(f"Batch image tensor shape: {images.shape}")
    print(f"Batch labels tensor shape: {labels.shape}")


100%|██████████| 26.4M/26.4M [00:01<00:00, 17.2MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 278kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.05MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.43MB/s]


Train batches: 375
Validation batches: 94
Test batches: 79
Computed mean: 0.2860, std: 0.3530


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Batch image tensor shape: torch.Size([128, 1, 28, 28])
Batch labels tensor shape: torch.Size([128])


In [24]:
import numpy as np

def l2_loss(y, yh):
  return 0.5 * (yh - y)**2

def l2_loss_grad(y, yh):
  return yh - y

def cross_entropy(y, yh):
  return -np.sum(y * np.log(yh + 1e-12))

# note that this is true only for dL/dz, L = loss(softmax(z))
def cross_entropy_grad(y, yh):
  return yh - y

def relu(x):
  return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    s = sigmoid(x)
    return s * (1 - s)
  
def tanh(x):
  return np.tanh(x)

def tanh_grad(x):
  t = np.tanh(x)
  return 1 - t * t

def linear(x):
  return x

def linear_grad(x):
  return np.ones_like(len(x))

def softmax(x):
  z = x - np.max(x)
  e = np.exp(z)
  return e / np.sum(e)

# MLP object:
# dims: list[int] - how many neurons in input, {hidden layers}, output
# activation_fns: list[relu|sigmoid|tanh|linear|softmax] - activation functions applied to EACH HIDDEN LAYER
# W: list[np.array(n_in x n_out)] - Random init with Normal dist.
# b: list[np.array(n_out)] - Random init with Normal dist.
# seed: float - For training reproducibility
class MLP:
  def __init__(self, dims, activation_fns, seed=None):
    self.dims = dims
    self.seed = seed
    if seed:
      np.random.seed(seed)

    dims_len = len(dims)
    activation_fns_len = len(activation_fns)
    if dims_len - 1 != activation_fns_len:
      dims_len = len(dims)
      raise RuntimeError(f"Length {dims_len} of dims does not match length {activation_fns_len} of activation_fns")
    
    self.activation_fns = activation_fns

    W_list = []
    b_list = []
    for i, dim in enumerate(dims[1:], start=1):
      W_list.append(np.random.normal(loc=0.0, scale=1.0, size=(dims[i - 1], dims[i])))
      b_list.append(np.zeros(dims[i]))
    
    self.W = W_list
    self.b = b_list

  def feed_forward(self, x):
    Z = []
    A = []
    z = None # intermediate var init
    a = np.array(x) # "input activation"
    # will contain list[x, Vx, Wf(Vx), ...]
    Z.append(x)
    # will contain list[x, fn(Vx), fn(Wf(Vx)), ...]
    A.append(x)
    for i, (W, b, fn) in enumerate(zip(self.W, self.b, self.activation_fns), start=1):
      z = a.T @ W + b
      Z.append(z)
      a = fn(z)
      A.append(a)
      z = a

    return Z, A
  
  # def train(self, X, Y, batch_size, lr, epochs):
  #   deltas = [] # deltas will be in same order as list of weight matrices/bias vectors
  #   N = X.shape[0]
  #   d = X.shape[1]
  #   num_layers = len(self.dims)
  #   train_batches = np.array_split(list(zip(X, Y)), N // batch_size)
  #   for epoch in range(epochs):
  #     for batch in train_batches:
  #       weight_update_per_batch = []
  #       bias_updates_per_batch = []
  #       for x, y in batch:
  #         deltas = []
  #         Z, A = self.feed_forward(x)

  #         last_activation_fn = self.activation_fns[len(self.activation_fns) - 1]
  #         # classification output
  #         if last_activation_fn is softmax:
  #           delta_last = A[num_layers - 1] - y
  #         # regression output
  #         else:
  #           delta_last = l2_loss_grad(A[num_layers - 1]) * activation_to_grad_map[last_activation_fn](Z[num_layers - 1])
          
  #         deltas.insert(0, delta_last)
          
  #         # move backwards (BACKprop)
  #         # note that self.W and self.activation_fns (to_second_layer, to_third_layer, ...) are 1 shorter than Z (list[x, Vx, Wf(Vx), ...])
  #         for layer in range(num_layers - 2, 0, -1):
  #           activation_fn = self.activation_fns[layer]
  #           delta = (self.W[layer].T @ deltas[0]) * activation_to_grad_map[activation_fn](Z[layer])
  #           deltas.insert(0, delta)
          
  #         weight_updates_per_instance = []
  #         bias_updates_per_instance = []
  #         for i, delta in enumerate(deltas):
  #           grad = A[i] @ delta
  #           weight_updates_per_instance.append(grad / np.linalg.norm(grad))
  #           bias_updates_per_instance.append(delta / np.linalg.norm(delta))
          
  #         weight_update_per_batch.append(weight_updates_per_instance)
  #         bias_updates_per_batch.append(bias_updates_per_instance)
        
  #       for i, (selfW, selfB) in enumerate(self.W, self.b):
  #         for (weight_batch_update, bias_batch_update) in zip(weight_update_per_batch, bias_updates_per_instance):
  #           selfW -= lr * weight_batch_update[i]
  #           selfB -= lr *bias_batch_update[i]
          
  #         self.W[i] = selfW
  #         self.b[i] = selfB

  def train(self, X, Y, batch_size, lr, epochs):
    """
    - Supports SGD/minibatch/full-batch via batch_size.
    - Softmax+CE: δ_L = y_hat - y (no extra f').
    - MSE (or any other): δ_L = (y_hat - y) ⊙ f'(z_L).
    Assumes feed_forward(x) -> (Z, A) with:
      A[0] = x, Z[1..L], A[L] = y_hat
    """
    activation_to_grad_map = {
        relu: relu_grad,
        sigmoid: sigmoid_grad,
        tanh: tanh_grad,
        linear: linear_grad,
    }
    
    N = X.shape[0]
    L = len(self.dims) - 1  # number of weight layers

    for epoch in range(epochs):
        print(f"Processing epoch {epoch + 1}...") if epoch % 10 == 0 else None
        # shuffle each epoch
        perm = np.random.permutation(N)
        Xs, Ys = X[perm], Y[perm]

        for start in range(0, N, batch_size):
            end = min(start + batch_size, N)
            m = end - start

            # gradient accumulators
            grad_W = [np.zeros_like(W) for W in self.W]
            grad_b = [np.zeros_like(b) for b in self.b]

            # per-sample backprop, sum then average
            for x, y in zip(Xs[start:end], Ys[start:end]):
                Z, A = self.feed_forward(x)  # A[0]=x, A[L]=ŷ; Z[1..L]
                last_act = self.activation_fns[-1]

                # delta per layer index: delta_layers[l] is deltal, for l=1..L. Means the array of deltas exactly match
                # layers (means 0th delta is None because that's for input.)
                delta_layers = [None] * (L + 1)

                # Output layer delta
                if last_act is softmax:
                    delta_layers[L] = A[-1] - y
                else:
                    dz_L = activation_to_grad_map[last_act](Z[-1])  # f'(z_L)
                    delta_layers[L] = (A[-1] - y) * dz_L

                # Hidden layers: l = L-1 .. 1
                for l in range(L - 1, 0, -1):
                    act = self.activation_fns[l - 1]           # activation after W[l-1]
                    dz = activation_to_grad_map[act](Z[l])      # f'(z_l)
                    
                    delta_layers[l] = (self.W[l] @ delta_layers[l + 1]) * dz

                # Gradients for each weight layer i=0..L-1 (maps layer i -> i+1)
                for i in range(L):
                    # ∂L/∂W[i] = A[i] (col) ⊗ δ^{i+1} (row)
                    grad_W[i] += np.outer(A[i], delta_layers[i + 1])
                    # ∂L/∂b[i] = δ^{i+1}
                    grad_b[i] += delta_layers[i + 1]

            # Apply averaged batch gradients
            inv_m = 1.0 / m
            for i in range(L):
                self.W[i] -= lr * (grad_W[i] * inv_m)
                self.b[i] -= lr * (grad_b[i] * inv_m)
        
        print(self.loss(np.array([1,2,3,4,5]), np.array([1,1,1]))) if epoch % 10 == 0 else None
  
  def loss(self, x, y):
    last_act = self.activation_fns[-1]
    Z, A = self.feed_forward(x)
    
    # CE loss
    if last_act is softmax:
      loss = cross_entropy(y, A[-1])
    # L2 loss
    else:
      loss = l2_loss(y, A[-1])
    
    return loss
     

mlp = MLP([5,10,10,3], [sigmoid, linear, softmax])
mlp.train(np.array([np.array([1,2,3,4,5])]), np.array(np.array([1,1,1])), 1, 0.001, 100)




Processing epoch 1...
14.971689027023572
Processing epoch 11...
12.747667704691425
Processing epoch 21...
10.718051934971527
Processing epoch 31...
9.063211893814346
Processing epoch 41...
7.936221010728584
Processing epoch 51...
7.26028014209837
Processing epoch 61...
6.824966003114476
Processing epoch 71...
6.479547909048205
Processing epoch 81...
6.154880555780511
Processing epoch 91...
5.825663412492704
